In [6]:
import pandas as pd
import runs
import numpy as np
from pathlib import Path
from IPython.display import IFrame
import pandas as pd 
import os
import shutil
import datetime

path = "Resources/"

In [7]:
def save(contents,name):
    f=open(path+name+".js","w+")
    f.write(contents)
    f.close()

    f=open(path+name+".html", "w+")
    f.write('<script src="https://code.highcharts.com/highcharts.js"></script>'+ "\n")
    f.write('<script src="https://code.highcharts.com/modules/series-label.js"></script>'+ "\n")
    f.write('<script src="https://code.highcharts.com/modules/exporting.js"></script>'+ "\n")
    f.write('<script src="https://code.highcharts.com/modules/export-data.js"></script>'+ "\n")
    f.write('<script src="https://code.highcharts.com/modules/parallel-coordinates.js"></script>'+ "\n\n")

    f.write("<div id='container'></div>"+ "\n")
    f.write("<script src='"+name+".js'></script>")
    f.close() 

In [8]:
def map_rules(ds,vs,categ):
    rule = {}
    i = 0
    for v in vs:
        rule[v] = i
        i += 1
    ds[categ] = ds[categ].map(rule)
    return ds

def parser(r_pattern):
    pat = r_pattern.split(" ")
    pattern = {"name": pat[0], "operator":pat[1], "value": pat[2]}
    return pattern

def multi_parser(r_patterns):
    patterns = []
    for pattern in r_patterns:
        patterns.append(parser(pattern))
    return patterns

def dividePat(pattern):
    ptrns = pattern.split(" AND ")
    rptr = []
    for ptr in ptrns:
        rptr.append(parser(ptr))
    
    return rptr,ptrns

def query_pattern(options):
    return "{} {} {}".format(options["name"],options["operator"],options["value"])

def n_query_pattern(options):
    op = ""
    if options["operator"] == "==":
        op = "!="
    else:
        op = "=="
    return "{} {} {}".format(options["name"],op,options["value"])

In [9]:
def run3y3(data,labels,categ,name,fname):    
    contents = Path(path+"template3y3.js").read_text()
    multis = "{name: '#name#',data: #data#,shadow: false,},"
    
    series = ""
    for ind in range(len(data)):

        d = data[ind]
            
        datastr = ""
        datastr = multis.replace("#name#",str(ind))
        datastr = datastr.replace("#data#",str(d.tolist()))
        
        series+= datastr
    
    contents = contents.replace("#series#",series)
    contents = contents.replace("#categ#",str(categ))
    contents = contents.replace("#classes#",str(labels))
    contents = contents.replace("#titulo#",""+name+"")
    
    save(contents,fname)
    
def ThreeCatThreeNum(patterns,dataset,name,fname):
    
    ptrns,fptrns = dividePat(patterns)
    
    # Numerical
    options1 = ptrns[0]
    options2 = ptrns[1]
    options3 = ptrns[2]
    
    # Categories 
    categ1 = ptrns[3]
    categ2 = ptrns[4]
    categ3 = ptrns[5]
        
    ds = dataset[[options1["name"],
                    options2["name"],
                    options3["name"],
                    categ1["name"],
                    categ2["name"],
                    categ3["name"],
                    "class"]]
    
    clss = np.unique(ds["class"]).tolist()
    ds = map_rules(ds,clss,"class")
    
    vs1 = np.unique(ds[categ1["name"]]).tolist()
    ds = map_rules(ds,vs1,categ1["name"])
    
    vs2 = np.unique(ds[categ2["name"]]).tolist()
    ds = map_rules(ds,vs2,categ2["name"])
    
    vs3 = np.unique(ds[categ3["name"]]).tolist()
    ds = map_rules(ds,vs3,categ3["name"])

    return ds,fptrns,clss

In [10]:
dt3 = pd.read_csv(path+"online_shoppers_intention.csv") 
dt3 = dt3.rename(columns={"Revenue": "class"})
data = dt3.iloc[:200,:]

In [11]:
data,patterns,classes = ThreeCatThreeNum("BounceRates > 0.0 AND Administrative_Duration < 50 AND ProductRelated_Duration < 100 AND Month == Feb AND VisitorType == Returning_Visitor AND OperatingSystems == 1",
                 data,"NumericalCategorical 4Item Binary","AlloftheAbove")

/Users/mackaber/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [12]:
run3y3(data.values,classes,patterns,"3 Cat y 3 Num","AlloftheAbove")

In [13]:
IFrame(src=path+'AlloftheAbove.html', width=650, height=650)